# Feature Engineering

ForecastFlowML includes a preprocessing module to create features bas ed on the time 
series dataset. This user guide shows how the features can be created in a scaleable way
before the modelling phase.

## Imports

In [1]:
from forecastflowml import FeatureExtractor
from forecastflowml import ForecastFlowML
from forecastflowml.data.loader import load_walmart_m5
from pyspark.sql import SparkSession
from lightgbm import LGBMRegressor
import pandas as pd

pd.set_option("display.max_columns", 100)

## Initialize Spark

In [2]:
spark = (
    SparkSession.builder.master("local[1]")
    .config("spark.driver.memory", "2g")
    .config("spark.sql.shuffle.partitions", "1")
    .config("spark.sql.execution.arrow.enabled", "true")
    .getOrCreate()
)

## Sample Dataset

In [3]:
df = load_walmart_m5(spark).localCheckpoint()
df.show(10)

+--------------------+-----------+-------+------+--------+--------+----------+-----+
|                  id|    item_id|dept_id|cat_id|store_id|state_id|      date|sales|
+--------------------+-----------+-------+------+--------+--------+----------+-----+
|FOODS_1_013_TX_2_...|FOODS_1_013|FOODS_1| FOODS|    TX_2|      TX|2015-01-01|  0.0|
|FOODS_1_013_TX_2_...|FOODS_1_013|FOODS_1| FOODS|    TX_2|      TX|2015-01-02|  0.0|
|FOODS_1_013_TX_2_...|FOODS_1_013|FOODS_1| FOODS|    TX_2|      TX|2015-01-03|  0.0|
|FOODS_1_013_TX_2_...|FOODS_1_013|FOODS_1| FOODS|    TX_2|      TX|2015-01-04|  0.0|
|FOODS_1_013_TX_2_...|FOODS_1_013|FOODS_1| FOODS|    TX_2|      TX|2015-01-05|  0.0|
|FOODS_1_013_TX_2_...|FOODS_1_013|FOODS_1| FOODS|    TX_2|      TX|2015-01-06|  0.0|
|FOODS_1_013_TX_2_...|FOODS_1_013|FOODS_1| FOODS|    TX_2|      TX|2015-01-07|  0.0|
|FOODS_1_013_TX_2_...|FOODS_1_013|FOODS_1| FOODS|    TX_2|      TX|2015-01-08|  0.0|
|FOODS_1_013_TX_2_...|FOODS_1_013|FOODS_1| FOODS|    TX_2|      T

## Feature Overview


With ``FeatureExtractor``, we can extract:
- Lag features
- Rolling statistics (mean, standard deviation etc.) with spesified lags
- Count of consecutive spesific values that may be used to count number of out-of-stock periods
- History length that refers to the number of periods from the beginning of the time series
- Date features


## Lags

When extracting the features, we should be careful about the lags we are creating.
In this example, we are going to prepare features for 4 weekly models.

- Model 1 will predict days 1–7, not using the the 6 most recent lag features.
- Model 2 will predict days 8–14, not using the the 13 most recent lag features.
- Model 3 will predict dayts 15–21, not using the the 20 most recent lag features.
- Model 4 will predict days 22–28, not using the the 27 most recent lag features.

For lag features, we are going to extract the sales on the same week day over the past 4 weeks. 

![image info](../_static/lag.svg)

Since each model has different horizon, they will be allowed to use different lags in the modelling phase. In summary, we need to extract ``lag_7``, ``lag_14``, ``lag_21``, ``lag_28``, ``lag_35``, ``lag_42`` and ``lag_49`` as features.

In [4]:
feature_extractor = FeatureExtractor(
    id_col="id",
    date_col="date",
    target_col="sales",
    lag_window_features={
        "lag": [7 * (i + 1) for i in range(8)],
    },
)
df_features = feature_extractor.transform(df)
df_features.show(10)

+--------------------+-----------+-------+------+--------+--------+----------+-----+-----+------+------+------+------+------+------+------+
|                  id|    item_id|dept_id|cat_id|store_id|state_id|      date|sales|lag_7|lag_14|lag_21|lag_28|lag_35|lag_42|lag_49|lag_56|
+--------------------+-----------+-------+------+--------+--------+----------+-----+-----+------+------+------+------+------+------+------+
|FOODS_1_002_TX_1_...|FOODS_1_002|FOODS_1| FOODS|    TX_1|      TX|2015-01-01|  0.0| null|  null|  null|  null|  null|  null|  null|  null|
|FOODS_1_002_TX_1_...|FOODS_1_002|FOODS_1| FOODS|    TX_1|      TX|2015-01-02|  0.0| null|  null|  null|  null|  null|  null|  null|  null|
|FOODS_1_002_TX_1_...|FOODS_1_002|FOODS_1| FOODS|    TX_1|      TX|2015-01-03|  0.0| null|  null|  null|  null|  null|  null|  null|  null|
|FOODS_1_002_TX_1_...|FOODS_1_002|FOODS_1| FOODS|    TX_1|      TX|2015-01-04|  0.0| null|  null|  null|  null|  null|  null|  null|  null|
|FOODS_1_002_TX_1_..

## Rolling Statistics

For rolling statistics, we are going to calculate the mean over the **window** of 7, 14 and 30 days, with the **most recent lags** that models can use which are 7 days for model 1, 14 days for model 2, 21 days for model 3 and 28 days for model 4.

![image info](../_static/lag_window.svg)

In [5]:
feature_extractor = FeatureExtractor(
    id_col="id",
    date_col="date",
    target_col="sales",
    lag_window_features={
        "mean": [[window, lag] for lag in [7, 14, 21, 28] for window in [7, 14, 30]],
    },
)
df_features = feature_extractor.transform(df)
df_features.show(10)

+--------------------+-----------+-------+------+--------+--------+----------+-----+-------------------+--------------------+--------------------+--------------------+---------------------+---------------------+--------------------+---------------------+---------------------+--------------------+---------------------+---------------------+
|                  id|    item_id|dept_id|cat_id|store_id|state_id|      date|sales|window_7_lag_7_mean|window_14_lag_7_mean|window_30_lag_7_mean|window_7_lag_14_mean|window_14_lag_14_mean|window_30_lag_14_mean|window_7_lag_21_mean|window_14_lag_21_mean|window_30_lag_21_mean|window_7_lag_28_mean|window_14_lag_28_mean|window_30_lag_28_mean|
+--------------------+-----------+-------+------+--------+--------+----------+-----+-------------------+--------------------+--------------------+--------------------+---------------------+---------------------+--------------------+---------------------+---------------------+--------------------+-------------------

## Out-of-stock Periods

Sometimes a product might be out-of-stock for a certain period. We are now going to
count the consecutive periods where sales did not occur with the **most recent lags** 
that models can use.

In [6]:
feature_extractor = FeatureExtractor(
    id_col="id",
    date_col="date",
    target_col="sales",
    count_consecutive_values={
        "value": 0,
        "lags": [7, 14, 21, 28],
    },
)
df_features = feature_extractor.transform(df)
df_features.show(10)

+--------------------+-----------+-------+------+--------+--------+----------+-----+-----------------------------+------------------------------+------------------------------+------------------------------+
|                  id|    item_id|dept_id|cat_id|store_id|state_id|      date|sales|count_consecutive_value_lag_7|count_consecutive_value_lag_14|count_consecutive_value_lag_21|count_consecutive_value_lag_28|
+--------------------+-----------+-------+------+--------+--------+----------+-----+-----------------------------+------------------------------+------------------------------+------------------------------+
|FOODS_1_002_TX_1_...|FOODS_1_002|FOODS_1| FOODS|    TX_1|      TX|2015-01-01|  0.0|                         null|                          null|                          null|                          null|
|FOODS_1_002_TX_1_...|FOODS_1_002|FOODS_1| FOODS|    TX_1|      TX|2015-01-02|  0.0|                         null|                          null|                       

## History Length

We can also count the total number periods past after the introduction of the time series.

In [7]:
feature_extractor = FeatureExtractor(
    id_col="id",
    date_col="date",
    target_col="sales",
    history_length=True,
)
df_features = feature_extractor.transform(df)
df_features.show(10)

+--------------------+-----------+-------+------+--------+--------+----------+-----+--------------+
|                  id|    item_id|dept_id|cat_id|store_id|state_id|      date|sales|history_length|
+--------------------+-----------+-------+------+--------+--------+----------+-----+--------------+
|FOODS_1_002_TX_1_...|FOODS_1_002|FOODS_1| FOODS|    TX_1|      TX|2015-01-01|  0.0|             1|
|FOODS_1_002_TX_1_...|FOODS_1_002|FOODS_1| FOODS|    TX_1|      TX|2015-01-02|  0.0|             2|
|FOODS_1_002_TX_1_...|FOODS_1_002|FOODS_1| FOODS|    TX_1|      TX|2015-01-03|  0.0|             3|
|FOODS_1_002_TX_1_...|FOODS_1_002|FOODS_1| FOODS|    TX_1|      TX|2015-01-04|  0.0|             4|
|FOODS_1_002_TX_1_...|FOODS_1_002|FOODS_1| FOODS|    TX_1|      TX|2015-01-05|  0.0|             5|
|FOODS_1_002_TX_1_...|FOODS_1_002|FOODS_1| FOODS|    TX_1|      TX|2015-01-06|  0.0|             6|
|FOODS_1_002_TX_1_...|FOODS_1_002|FOODS_1| FOODS|    TX_1|      TX|2015-01-07|  0.0|             7|


## Date Features

Finally, we can also include the date derived features.

In [8]:
feature_extractor = FeatureExtractor(
    id_col="id",
    date_col="date",
    target_col="sales",
    date_features=[
        "day_of_month",
        "day_of_week",
        "week_of_year",
        "week_of_month",
        "weekend",
        "quarter",
        "month",
        "year",
    ],
)
df_features = feature_extractor.transform(df)
df_features.show(10)

+--------------------+-----------+-------+------+--------+--------+----------+-----+------------+-----------+------------+-------------+-------+-------+-----+----+
|                  id|    item_id|dept_id|cat_id|store_id|state_id|      date|sales|day_of_month|day_of_week|week_of_year|week_of_month|weekend|quarter|month|year|
+--------------------+-----------+-------+------+--------+--------+----------+-----+------------+-----------+------------+-------------+-------+-------+-----+----+
|FOODS_1_013_TX_2_...|FOODS_1_013|FOODS_1| FOODS|    TX_2|      TX|2015-01-01|  0.0|           1|          5|           1|            1|      0|      1|    1|2015|
|FOODS_1_013_TX_2_...|FOODS_1_013|FOODS_1| FOODS|    TX_2|      TX|2015-01-02|  0.0|           2|          6|           1|            1|      0|      1|    1|2015|
|FOODS_1_013_TX_2_...|FOODS_1_013|FOODS_1| FOODS|    TX_2|      TX|2015-01-03|  0.0|           3|          7|           1|            1|      1|      1|    1|2015|
|FOODS_1_013_TX_

## Combine Features

Let's combine all of the features extraction steps together.

In [9]:
feature_extractor = FeatureExtractor(
    id_col="id",
    date_col="date",
    target_col="sales",
    lag_window_features={
        "lag": [7 * (i + 1) for i in range(8)],
        "mean": [[window, lag] for lag in [7, 14, 21, 28] for window in [7, 14, 30]],
    },
    date_features=[
        "day_of_month",
        "day_of_week",
        "week_of_year",
        "week_of_month",
        "weekend",
        "quarter",
        "month",
        "year",
    ],
    count_consecutive_values={
        "value": 0,
        "lags": [7, 14, 21, 28],
    },
    history_length=True,
)

### PySpark DataFrame

In [10]:
df_train = feature_extractor.transform(df).localCheckpoint()
df_train.show(10)

+--------------------+-----------+-------+------+--------+--------+----------+-----+-----+------+------+------+------+------+------+------+-------------------+--------------------+--------------------+--------------------+---------------------+---------------------+--------------------+---------------------+---------------------+--------------------+---------------------+---------------------+-----------------------------+------------------------------+------------------------------+------------------------------+--------------+------------+-----------+------------+-------------+-------+-------+-----+----+
|                  id|    item_id|dept_id|cat_id|store_id|state_id|      date|sales|lag_7|lag_14|lag_21|lag_28|lag_35|lag_42|lag_49|lag_56|window_7_lag_7_mean|window_14_lag_7_mean|window_30_lag_7_mean|window_7_lag_14_mean|window_14_lag_14_mean|window_30_lag_14_mean|window_7_lag_21_mean|window_14_lag_21_mean|window_30_lag_21_mean|window_7_lag_28_mean|window_14_lag_28_mean|window_30_la

### Pandas DataFrame

In [11]:
feature_extractor.transform(df.toPandas(), spark=spark)

,id,item_id,dept_id,cat_id,store_id,state_id,date,sales,lag_7,lag_14,lag_21,lag_28,lag_35,lag_42,lag_49,lag_56,window_7_lag_7_mean,window_14_lag_7_mean,window_30_lag_7_mean,window_7_lag_14_mean,window_14_lag_14_mean,window_30_lag_14_mean,window_7_lag_21_mean,window_14_lag_21_mean,window_30_lag_21_mean,window_7_lag_28_mean,window_14_lag_28_mean,window_30_lag_28_mean,count_consecutive_value_lag_7,count_consecutive_value_lag_14,count_consecutive_value_lag_21,count_consecutive_value_lag_28,history_length,day_of_month,day_of_week,week_of_year,week_of_month,weekend,quarter,month,year
0,FOODS_1_002_TX_1_evaluation,FOODS_1_002,FOODS_1,FOODS,TX_1,TX,2015-01-01,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,5,1,1,0,1,1,2015
1,FOODS_1_002_TX_1_evaluation,FOODS_1_002,FOODS_1,FOODS,TX_1,TX,2015-01-02,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,6,1,1,0,1,1,2015
2,FOODS_1_002_TX_1_evaluation,FOODS_1_002,FOODS_1,FOODS,TX_1,TX,2015-01-03,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,3,7,1,1,1,1,1,2015
3,FOODS_1_002_TX_1_evaluation,FOODS_1_002,FOODS_1,FOODS,TX_1,TX,2015-01-04,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,4,1,1,1,1,1,1,2015
4,FOODS_1_002_TX_1_evaluation,FOODS_1_002,FOODS_1,FOODS,TX_1,TX,2015-01-05,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,5,2,2,1,0,1,1,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252403,HOUSEHOLD_2_516_WI_2_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_2,WI,2016-05-18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.033333,0.0,0.071429,0.033333,0.142857,0.071429,0.033333,0.000000,0.000000,0.000000,20.0,13.0,6.0,41.0,504,18,4,20,3,0,2,5,2016
252404,HOUSEHOLD_2_516_WI_2_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_2,WI,2016-05-19,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.033333,0.0,0.000000,0.033333,0.000000,0.071429,0.033333,0.142857,0.071429,0.033333,21.0,14.0,7.0,0.0,505,19,5,20,3,0,2,5,2016
252405,HOUSEHOLD_2_516_WI_2_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_2,WI,2016-05-20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.033333,0.0,0.000000,0.033333,0.000000,0.071429,0.033333,0.142857,0.071429,0.033333,22.0,15.0,8.0,1.0,506,20,6,20,3,0,2,5,2016
252406,HOUSEHOLD_2_516_WI_2_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_2,WI,2016-05-21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.033333,0.0,0.000000,0.033333,0.000000,0.071429,0.033333,0.142857,0.071429,0.033333,23.0,16.0,9.0,2.0,507,21,7,20,3,1,2,5,2016


## Training

We can not pass the features created by ``FeatureExtractor`` to ``ForecastFlowML`` for training. As mentioned in the lag feature creation step, we are going to set ``use_lag_range=28`` to use lags which are 28 days after from the most recent lag features. 

In [12]:
forecast_flow = ForecastFlowML(
    group_col="store_id",
    id_col="id",
    date_col="date",
    target_col="sales",
    date_frequency="days",
    model_horizon=7,
    max_forecast_horizon=28,
    model=LGBMRegressor(),
    use_lag_range=28,
)
trained_models = forecast_flow.train(df_train).toPandas()
trained_models

,group,forecast_horizon,model,start_time,end_time,elapsed_seconds
0,CA_1,"[[1, 2, 3, 4, 5, 6, 7], [8, 9, 10, 11, 12, 13,...",[b'\x80\x04\x95\xec\x01\x00\x00\x00\x00\x00\x0...,10-May-2023 (17:10:07),10-May-2023 (17:10:08),0.8
1,CA_2,"[[1, 2, 3, 4, 5, 6, 7], [8, 9, 10, 11, 12, 13,...",[b'\x80\x04\x95\xec\x01\x00\x00\x00\x00\x00\x0...,10-May-2023 (17:10:08),10-May-2023 (17:10:09),0.6
2,CA_3,"[[1, 2, 3, 4, 5, 6, 7], [8, 9, 10, 11, 12, 13,...",[b'\x80\x04\x95\xec\x01\x00\x00\x00\x00\x00\x0...,10-May-2023 (17:10:09),10-May-2023 (17:10:10),0.6
3,CA_4,"[[1, 2, 3, 4, 5, 6, 7], [8, 9, 10, 11, 12, 13,...",[b'\x80\x04\x95\xec\x01\x00\x00\x00\x00\x00\x0...,10-May-2023 (17:10:10),10-May-2023 (17:10:10),0.7
4,TX_1,"[[1, 2, 3, 4, 5, 6, 7], [8, 9, 10, 11, 12, 13,...",[b'\x80\x04\x95\xec\x01\x00\x00\x00\x00\x00\x0...,10-May-2023 (17:10:10),10-May-2023 (17:10:11),0.5
5,TX_2,"[[1, 2, 3, 4, 5, 6, 7], [8, 9, 10, 11, 12, 13,...",[b'\x80\x04\x95\xec\x01\x00\x00\x00\x00\x00\x0...,10-May-2023 (17:10:11),10-May-2023 (17:10:12),0.5
6,TX_3,"[[1, 2, 3, 4, 5, 6, 7], [8, 9, 10, 11, 12, 13,...",[b'\x80\x04\x95\xec\x01\x00\x00\x00\x00\x00\x0...,10-May-2023 (17:10:12),10-May-2023 (17:10:12),0.5
7,WI_1,"[[1, 2, 3, 4, 5, 6, 7], [8, 9, 10, 11, 12, 13,...",[b'\x80\x04\x95\xec\x01\x00\x00\x00\x00\x00\x0...,10-May-2023 (17:10:12),10-May-2023 (17:10:13),0.5
8,WI_2,"[[1, 2, 3, 4, 5, 6, 7], [8, 9, 10, 11, 12, 13,...",[b'\x80\x04\x95\xec\x01\x00\x00\x00\x00\x00\x0...,10-May-2023 (17:10:13),10-May-2023 (17:10:13),0.5
9,WI_3,"[[1, 2, 3, 4, 5, 6, 7], [8, 9, 10, 11, 12, 13,...",[b'\x80\x04\x95\xec\x01\x00\x00\x00\x00\x00\x0...,10-May-2023 (17:10:13),10-May-2023 (17:10:14),0.5


### Examine Features

Let's examine which features are used for each model.

In [13]:
import pickle

features = {}
for i in range(4):
    model = pickle.loads(trained_models["model"].iloc[0][i])
    features[f"model_{i}"] = sorted(model.feature_name_)
pd.DataFrame(features)

,model_0,model_1,model_2,model_3
0,count_consecutive_value_lag_7,count_consecutive_value_lag_14,count_consecutive_value_lag_21,count_consecutive_value_lag_28
1,day_of_month,day_of_month,day_of_month,day_of_month
2,day_of_week,day_of_week,day_of_week,day_of_week
3,history_length,history_length,history_length,history_length
4,lag_14,lag_14,lag_21,lag_28
5,lag_21,lag_21,lag_28,lag_35
6,lag_28,lag_28,lag_35,lag_42
7,lag_35,lag_35,lag_42,lag_49
8,lag_7,lag_42,lag_49,lag_56
9,month,month,month,month
